In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException

import pandas as pd
import time



In [2]:
def scrape_samsung_life_products_on_sale(driver):
    data0 = []
    wait = WebDriverWait(driver, 30)

    def get_pdf_url(cell):
        try:
            pdf_link = cell.find_element(By.CSS_SELECTOR, "a.btn-file.icon-pdf")
            if pdf_link and pdf_link.is_displayed():
                main_window = driver.current_window_handle
                driver.execute_script("arguments[0].click();", pdf_link)
                time.sleep(1)
                if len(driver.window_handles) > 1:
                    driver.switch_to.window(driver.window_handles[-1])
                    url = driver.current_url
                    driver.close()
                    driver.switch_to.window(main_window)
                    return url
            return 'X'
        except Exception:
            return 'X'

    try:
        driver.get("https://www.samsunglife.com/individual/products/disclosure/sales/PDO-PRPRI010110M")
        print("삼성생명 공시실 상품공시 웹사이트에 접속했습니다.")
        time.sleep(3)

        tabs_to_process = ['판매상품', '판매중지상품']
        tab_content_map = {
            '판매상품': 'content1',
            '판매중지상품': 'content2'
        }

        # tabs_to_process = ['판매중지상품']
        # tab_content_map = {
        #     '판매중지상품': 'content2'
        # }

        for tab_name in tabs_to_process:
            print(f"\n{tab_name} 탭 처리 시작...")
            
            try:
                # 탭 선택 및 컨텐츠 로드
                tab = wait.until(EC.element_to_be_clickable((
                    By.XPATH, 
                    f"//ul[@class='tabs-group']/li/a[normalize-space()='{tab_name}']"
                )))
                driver.execute_script("arguments[0].click();", tab)
                time.sleep(1)

                content_id = tab_content_map[tab_name]
                
                while True:  # 페이지 순환
                    try:
                        # 현재 페이지의 tbody 찾기
                        tbody_xpath = f"//div[@id='{content_id}']//tbody[@class='line']"
                        tbody = wait.until(EC.presence_of_element_located((By.XPATH, tbody_xpath)))
                        rows = tbody.find_elements(By.TAG_NAME, 'tr')
                        print(f"현재 페이지에서 {len(rows)}개의 행을 찾았습니다.")

                        # print(f"rows: {rows}")

                        # 현재 페이지의 데이터 처리
                        for row in rows:
                            try:
                                cells = row.find_elements(By.TAG_NAME, 'td')
                                if len(cells) < 7:
                                    continue

                                record_num = cells[0].text.strip()
                                current_record = [
                                    record_num,
                                    "삼성생명",
                                    "판매중" if tab_name == "판매상품" else "판매중지",
                                    cells[1].text.strip(),
                                    cells[2].find_element(By.CSS_SELECTOR, '.title-link, a').text.strip(),
                                    cells[3].text.strip(),
                                    get_pdf_url(cells[4]) if tab_name == "판매상품" else 'X',
                                    get_pdf_url(cells[5]),
                                    get_pdf_url(cells[6])
                                ]
                                data0.append(current_record)
                                print(f"레코드 {record_num} 처리 완료")

                            except Exception as e:
                                print(f"행 처리 중 오류: {e}")
                                continue

                        # 페이지네이션 처리
                        pagination = wait.until(EC.presence_of_element_located((
                            By.XPATH, f"//div[@id='{content_id}']//div[@class='pagination-number']"
                        )))
                        
                        # 현재 페이지 번호 확인
                        current_page_text = pagination.find_element(By.CSS_SELECTOR, "li.current button").text
                        current_page = int(current_page_text.split()[0].strip())
                        print(f"현재 페이지: {current_page}")

                        # 다음 페이지 세트 버튼 상태 확인
                        next_set_button = wait.until(
                            EC.presence_of_element_located((
                                By.XPATH,
                                f"//div[@id='{content_id}']//button[@class='btn-paging-next']"
                            ))
                        )
                        is_last_set = next_set_button.get_attribute('disabled') == 'disabled'

                        if current_page % 10 == 0:  # 10, 20, 30 등 페이지 세트의 마지막
                            if is_last_set:
                                # 마지막 세트의 마지막 페이지인 경우
                                print(f"{tab_name} 탭의 모든 데이터 수집 완료")
                                time.sleep(2)  # 탭 전환 전 잠시 대기
                                break  # while 루프 종료
                            
                            print("다음 페이지 세트로 이동")
                            old_tbody = tbody
                            driver.execute_script("arguments[0].click();", next_set_button)
                            time.sleep(2)
                            
                            # 다음 세트의 첫 번째 페이지로 이동
                            first_page_of_set = ((current_page // 10) * 10) + 1
                            first_page_xpath = f"//div[@id='{content_id}']//div[@class='pagination-number']//li/button[text()='{first_page_of_set}']"
                            
                            try:
                                first_page_button = wait.until(
                                    EC.presence_of_element_located((By.XPATH, first_page_xpath))
                                )
                                driver.execute_script("arguments[0].click();", first_page_button)
                                time.sleep(2)
                                continue
                            except Exception as e:
                                print(f"첫 페이지 이동 중 오류: {e}")
                                break
                        else:
                            # 일반적인 다음 페이지 이동
                            try:
                                next_page = current_page + 1
                                next_button_xpath = f"//div[@id='{content_id}']//div[@class='pagination-number']//li[not(@class='current')]/button[text()='{next_page}']"
                                
                                try:
                                    next_button = wait.until(EC.presence_of_element_located((By.XPATH, next_button_xpath)))
                                    print(f"페이지 {next_page}로 이동")
                                    driver.execute_script("arguments[0].click();", next_button)
                                    time.sleep(2)
                                    continue
                                except TimeoutException:
                                    # 다음 버튼을 찾지 못한 경우
                                    if is_last_set:
                                        print(f"{tab_name}의 마지막 페이지입니다. 다음 탭으로 이동합니다.")
                                        break
                                    else:
                                        print("다음 페이지 세트로 이동이 필요합니다.")
                                        continue
                                    
                            except Exception as e:
                                if is_last_set:
                                    print(f"{tab_name}의 마지막 페이지입니다. 다음 탭으로 이동합니다.")
                                    break
                                else:
                                    print(f"페이지 이동 중 오류 발생: {e}")
                                    break

                    except Exception as e:
                        print(f"페이지 처리 중 오류: {e}")
                        if is_last_set:
                            print(f"{tab_name} 탭의 모든 데이터 수집 완료")
                            time.sleep(2)  # 탭 전환 전 잠시 대기
                            break  # while 루프 종료
                        break

            except Exception as e:
                print(f"탭 처리 중 오류: {e}")
                continue  # 다음 탭으로 이동

            # 현재 탭의 데이터 수집이 완료되면 진행 상황 출력
            print(f"\n=== {tab_name} 탭 처리 완료 ===")
            print(f"수집된 데이터 수: {len(data0)}")
            if tab_name != tabs_to_process[-1]:  # 마지막 탭이 아닌 경우
                print(f"다음 탭 '{tabs_to_process[tabs_to_process.index(tab_name) + 1]}' 처리 시작...\n")
                time.sleep(3)  # 탭 전환 전 충분한 대기 시간

    except Exception as e:
        print(f"크롤링 중 오류 발생: {e}")

    finally:
        return pd.DataFrame(data0, columns=[
            "품번", "판매사", "판매구분", "분류", "상품명", 
            "판매기간", "요약서", "방법서", "약관"
        ]), driver

In [3]:
def scrape_samsung_life_products_stop_sale(driver):
    data0 = []
    wait = WebDriverWait(driver, 30)

    def get_pdf_url(cell):
        try:
            pdf_link = cell.find_element(By.CSS_SELECTOR, "a.btn-file.icon-pdf")
            if pdf_link and pdf_link.is_displayed():
                main_window = driver.current_window_handle
                driver.execute_script("arguments[0].click();", pdf_link)
                time.sleep(1)
                if len(driver.window_handles) > 1:
                    driver.switch_to.window(driver.window_handles[-1])
                    url = driver.current_url
                    driver.close()
                    driver.switch_to.window(main_window)
                    return url
            return 'X'
        except Exception:
            return 'X'

    try:
        driver.get("https://www.samsunglife.com/individual/products/disclosure/sales/PDO-PRPRI010110M")
        print("삼성생명 공시실 상품공시 웹사이트에 접속했습니다.")
        time.sleep(3)

        # tabs_to_process = ['판매상품', '판매중지상품']
        # tab_content_map = {
        #     '판매상품': 'content1',
        #     '판매중지상품': 'content2'
        # }

        tabs_to_process = ['판매중지상품']
        tab_content_map = {
            '판매중지상품': 'content2'
        }

        for tab_name in tabs_to_process:
            print(f"\n{tab_name} 탭 처리 시작...")
            
            try:
                # 탭 선택 및 컨텐츠 로드
                tab = wait.until(EC.element_to_be_clickable((
                    By.XPATH, 
                    f"//ul[@class='tabs-group']/li/a[normalize-space()='{tab_name}']"
                )))
                driver.execute_script("arguments[0].click();", tab)
                time.sleep(1)

                content_id = tab_content_map[tab_name]
                
                while True:  # 페이지 순환
                    try:
                        # 현재 페이지의 tbody 찾기
                        tbody_xpath = f"//div[@id='{content_id}']//tbody[@class='line']"
                        tbody = wait.until(EC.presence_of_element_located((By.XPATH, tbody_xpath)))
                        rows = tbody.find_elements(By.TAG_NAME, 'tr')
                        print(f"현재 페이지에서 {len(rows)}개의 행을 찾았습니다.")

                        # print(f"rows: {rows}")

                        # 현재 페이지의 데이터 처리
                        for row in rows:
                            try:
                                cells = row.find_elements(By.TAG_NAME, 'td')
                                # print(f"셀 개수: {len(cells)}")  # 디버깅용
                                
                                if len(cells) < 6:  # 셀 개수 체크 수정 (7 -> 6)
                                    # print(f"셀 개수 부족: {len(cells)}")
                                    continue

                                record_num = cells[0].text.strip()
                                
                                # PDF URL 가져오기 (요약서 제외)
                                current_record = [
                                    record_num,
                                    "삼성생명",
                                    "판매중지",
                                    cells[1].text.strip(),
                                    cells[2].find_element(By.CSS_SELECTOR, '.title-link, a').text.strip(),
                                    cells[3].text.strip(),
                                    'X',  # 요약서는 항상 'X'로 설정
                                    get_pdf_url(cells[4]),  # 방법서
                                    get_pdf_url(cells[5])   # 약관
                                ]
                                
                                data0.append(current_record)
                                print(f"레코드 {record_num} 처리 완료")

                            except Exception as e:
                                print(f"행 처리 중 오류: {e}")
                                continue

                        # 페이지네이션 처리
                        pagination = wait.until(EC.presence_of_element_located((
                            By.XPATH, f"//div[@id='{content_id}']//div[@class='pagination-number']"
                        )))
                        
                        # 현재 페이지 번호 확인
                        current_page_text = pagination.find_element(By.CSS_SELECTOR, "li.current button").text
                        current_page = int(current_page_text.split()[0].strip())
                        print(f"현재 페이지: {current_page}")

                        # 다음 페이지 세트 버튼 상태 확인
                        next_set_button = wait.until(
                            EC.presence_of_element_located((
                                By.XPATH,
                                f"//div[@id='{content_id}']//button[@class='btn-paging-next']"
                            ))
                        )
                        is_last_set = next_set_button.get_attribute('disabled') == 'disabled'

                        if current_page % 10 == 0:  # 10, 20, 30 등 페이지 세트의 마지막
                            if is_last_set:
                                # 마지막 세트의 마지막 페이지인 경우
                                print("마지막 페이지입니다. 다음 탭으로 이동")
                                break
                            
                            print("다음 페이지 세트로 이동")
                            old_tbody = tbody
                            driver.execute_script("arguments[0].click();", next_set_button)
                            time.sleep(2)
                            
                            # 다음 세트의 첫 번째 페이지로 이동
                            first_page_of_set = ((current_page // 10) * 10) + 1
                            first_page_xpath = f"//div[@id='{content_id}']//div[@class='pagination-number']//li/button[text()='{first_page_of_set}']"
                            
                            try:
                                first_page_button = wait.until(
                                    EC.presence_of_element_located((By.XPATH, first_page_xpath))
                                )
                                driver.execute_script("arguments[0].click();", first_page_button)
                                time.sleep(2)
                                continue
                            except Exception as e:
                                print(f"첫 페이지 이동 중 오류: {e}")
                                break
                        else:
                            # 일반적인 다음 페이지 이동
                            try:
                                next_page = current_page + 1
                                next_button_xpath = f"//div[@id='{content_id}']//div[@class='pagination-number']//li[not(@class='current')]/button[text()='{next_page}']"
                                
                                try:
                                    next_button = wait.until(EC.presence_of_element_located((By.XPATH, next_button_xpath)))
                                    print(f"페이지 {next_page}로 이동")
                                    driver.execute_script("arguments[0].click();", next_button)
                                    time.sleep(2)
                                    continue
                                except TimeoutException:
                                    if is_last_set:
                                        print("마지막 페이지입니다. 다음 탭으로 이동")
                                        break
                                    raise
                                    
                            except Exception as e:
                                print(f"다음 페이지 이동 중 오류: {e}")
                                break

                    except Exception as e:
                        print(f"페이지 처리 중 오류: {e}")
                        break

            except Exception as e:
                print(f"탭 처리 중 오류: {e}")
                continue

    except Exception as e:
        print(f"크롤링 중 오류 발생: {e}")

    finally:
        return pd.DataFrame(data0, columns=[
            "품번", "판매사", "판매구분", "분류", "상품명", 
            "판매기간", "요약서", "방법서", "약관"
        ]), driver

In [4]:
from webdriver_manager.chrome import ChromeDriverManager
# 메인 실행 부분
chrome_options = Options()
chrome_options.add_argument("--headless")  # 헤드리스 모드
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriverManager가 자동으로 알맞은 버전의 드라이버를 설치합니다
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    df1, driver = scrape_samsung_life_products_on_sale(driver)
    # df1, driver = scrape_samsung_life_products_stop_sale(driver)
    
    # df1을 엑셀로 저장
    output_file = "samsung_life_products.xlsx"
    df1.to_excel(output_file, index=False)
    
    print(f"데이터를 {output_file} 파일로 저장했습니다.")
    print(f"총 {len(df1)} 개의 행이 저장되었습니다.")
    
    # 판매구분별 행 수 출력
    print("\n판매구분별 행 수:")
    print(df1['판매구분'].value_counts())
    
    # 판매사별 행 수 출력
    print("\n판매사별 행 수:")
    print(df1['판매사'].value_counts())

finally:
    driver.quit()
    print("브라우저를 종료했습니다.")

삼성생명 공시실 상품공시 웹사이트에 접속했습니다.

판매상품 탭 처리 시작...
현재 페이지에서 10개의 행을 찾았습니다.
레코드 293 처리 완료
레코드 292 처리 완료
레코드 291 처리 완료
레코드 290 처리 완료
레코드 289 처리 완료
레코드 288 처리 완료
레코드 287 처리 완료
레코드 286 처리 완료
레코드 285 처리 완료
레코드 284 처리 완료
현재 페이지: 1
페이지 2로 이동
현재 페이지에서 10개의 행을 찾았습니다.
레코드 283 처리 완료
레코드 282 처리 완료
레코드 281 처리 완료
레코드 280 처리 완료
레코드 279 처리 완료
레코드 278 처리 완료
레코드 277 처리 완료
레코드 276 처리 완료
레코드 275 처리 완료
레코드 274 처리 완료
현재 페이지: 2
페이지 3로 이동
현재 페이지에서 10개의 행을 찾았습니다.
레코드 273 처리 완료
레코드 272 처리 완료
레코드 271 처리 완료
레코드 270 처리 완료
레코드 269 처리 완료
레코드 268 처리 완료
레코드 267 처리 완료
레코드 266 처리 완료
레코드 265 처리 완료
레코드 264 처리 완료
현재 페이지: 3
페이지 4로 이동
현재 페이지에서 10개의 행을 찾았습니다.
레코드 263 처리 완료
레코드 262 처리 완료
